In [1]:
import pandas as pd

#### Reading graph from csv file. Storing it as a dictionary, i.e. adjacent list

In [48]:
import sys
import csv

csv.field_size_limit(sys.maxsize)
graph = {}
for key, val in csv.reader(open("data/graph.csv")):
    if val== "[]":
        graph[int(key)] = [] 
    else:
        graph[int(key)] = [int(n) for n in val[1:-1].split(',')]

In [49]:
user_id_1 = []
user_id_2 = []

In [50]:
for key in graph:
    for i in graph[key]:
        user_id_1.append(int(key))
        user_id_2.append(int(i))

In [51]:
graph_frame = pd.DataFrame(
    {'user_id_1': user_id_1,
     'user_id_2': user_id_2
    })

In [52]:
users_children = graph_frame.user_id_2.unique()

#### Making graph inverted to be able to see all the parents of the node

In [53]:
graph_inverted = {}
for i in users_children:
    graph_inverted[i] = \
    graph_frame.user_id_1[graph_frame.user_id_2 == i].tolist()

In [54]:
retwitts = pd.read_csv("data/small_retweets.csv", sep='\t',  index_col=0)
twitts = pd.read_csv("data/small_tweets.csv", sep='\t',  index_col=0)

In [55]:
retwitts.time = pd.to_datetime(retwitts.time)

In [56]:
twitts.time = pd.to_datetime(twitts.time)

In [57]:
len(retwitts.user_id.unique())

56390

#### Reading the list of frames where each frame is a log for particular twitt

In [58]:
import glob
path ='data/twitter_data' # use your path
allFiles = glob.glob(path + "/*.csv")
list_of_frames = {}
SHIFT = 1000000000.*60.*60.
def time_conv(x):
    return x.value/SHIFT
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=0, sep= '\t')
    df.time = pd.to_datetime(df.time)
    df.time = df.time.apply(time_conv)
    list_of_frames[int((file_.split('/')[2]).split('.')[0])] = df

In [59]:
twitts.time = twitts.time.apply(time_conv)

In [60]:
retwitts.time = retwitts.time.apply(time_conv)

#### For each frame calculating the maximum time 

In [61]:
max_times = {}

In [62]:
from datetime import timedelta, datetime, date
for key in list_of_frames:
    max_times[key] = list_of_frames[key].time.max()

In [63]:
for key in list_of_frames:
    list_of_frames[key] = list_of_frames[key].reset_index(drop=True)

In [64]:
retwitts = retwitts.reset_index(drop=True)
twitts = twitts.reset_index(drop=True)

In [65]:
twitts.head()

,user_id,tweet_id,time,num
0,1257818405,3449713464551829,371663.0,503265
1,1288915263,3470389738808369,373032.0,500152
2,2239954024,3418421884953768,369590.0,499253


In [66]:
del twitts["num"]

In [67]:
twitts

,user_id,tweet_id,time
0,1257818405,3449713464551829,371663.0
1,1288915263,3470389738808369,373032.0
2,2239954024,3418421884953768,369590.0


#### Mearging together the seed set entities and logs. Just because we need to have seed set log in the whole set of logs

In [239]:
#for index, row in twitts.iterrows():
#   print row
#  list_of_frames[row["tweet_id"]].loc[len(list_of_frames[row["tweet_id"]])] = row

#### For each node in each set calculating the time of activation. Making a dictionary

In [68]:
activ_times = {}
for key in list_of_frames:
    activ_times[key] = {}
    u_users = list_of_frames[key].user_id.unique() 
    for u in u_users:
        activ_times[key][u] = \
            list_of_frames[key].time[list_of_frames[key].user_id == u].min()
    seed_user = \
        twitts.user_id[twitts.tweet_id == key].tolist()[0]
    activ_times[key][seed_user] =\
        twitts.time[twitts.tweet_id == key].tolist()[0]

In [69]:
def activ_time(S, id):
    try:
        return activ_times[S][id]
    except:
        return 0

In [70]:
## constants declaration
DELTA = 24
ETA = 5

$$ p_{vw} \textrm{  inistialization} $$
$$ \textrm{Function for  }  P_{w}^{(s)} \textrm{ computation } $$

In [71]:
len(graph_frame)
import numpy as np
probs = np.random.random(len(graph_frame))
grad = np.zeros(len(graph_frame))

In [72]:
edge_index = {}
subset = graph_frame[["user_id_1", "user_id_2"]]
edges = [tuple(x) for x in subset.values]

In [73]:
j = 0
for i in edges:
    edge_index[i] = j
    j = j + 1

In [74]:
def P(w, S):
    res = 1
    w_act = activ_time(S, w)
    if w in graph_inverted:
        for tau in range(0, DELTA):
            for v in graph_inverted[w]:
                if (activ_time(S, v) == w_act - 1 - tau):
                    res= res * (1 - probs[edge_index[(v,w)]]*np.exp(-tau/ETA))
    return 1-res

In [75]:
P( 120671, 5615734448357004557 )

0

In [76]:
for key in list_of_frames:
    print list_of_frames[key].head()

      user_id          tweet_id      time
0  1767454430  3418421884953768  369615.0
1  1767467245  3418421884953768  369595.0
2  1767468271  3418421884953768  369615.0
3  1657168480  3418421884953768  369610.0
4  1767471057  3418421884953768  369609.0
      user_id          tweet_id      time
0       10057  3470389738808369  373040.0
1  1837296221  3470389738808369  373035.0
2  1767465991  3470389738808369  373046.0
3  1767464154  3470389738808369  373032.0
4  1767466527  3470389738808369  373051.0
      user_id          tweet_id      time
0  1767471542  3449713464551829  371665.0
1  1837335093  3449713464551829  371677.0
2  1767491242  3449713464551829  371674.0
3  1767510241  3449713464551829  371663.0
4  1678644414  3449713464551829  371694.0


### Computation of

$$
	\frac{\partial P_{w}^{(s)}}{\partial p_{vw}}
	= 
	\left({\displaystyle \prod_{\tau = 0}^{\Delta-1}\prod_{\substack{v' \in B(w) \cap D(t_w-\tau-1)\\v' \neq v}} (1-p_{v'w}e^{-\tau/\eta})}\right) e^{-(t_w-t_v-1)/\eta}
$$

In [77]:
def partial(S,v,w):
    w_act = activ_time(S, w)
    res = 1
    if w in graph_inverted:
        for tau in range(0,DELTA):
            for v_prime in graph_inverted[w]:
                if (activ_time(S, v_prime) == w_act - 1 - tau) and (v_prime != v):
                    res = res * (1 - probs[edge_index[(v_prime,w)]]*np.exp(-tau/ETA))           
    res = res*np.exp(-(w_act - activ_time(S, v) -1)/ETA)
    return res

In [78]:
partial(3445888074596137,1616700401,1066926080)

1.0

### Computation of

$$
	Z_{vw}
	=
	-\sum_{\tau = 0}^{t_w-t_v-2}\frac{1}{1 - p_{vw}e^{-\tau/\eta}} e^{-\tau/\eta}
$$

In [79]:
def Z(S, v, w):
    res = 0
    w_act = activ_time(S, w)
    v_act = activ_time(S, v)
    for tau in range(0, int(w_act - v_act - 1)):
        e = np.exp(-tau/ETA)
        res = res + e/(1 -  probs[edge_index[(v,w)]]*e)
    return res

In [80]:
Z(3445888074596137,1616700401,1066926080)

0

### Computation of gradient component

$$
	\frac{\partial L}{\partial p_{vw}} = \sum_{s=1}^{S}
	\left\{
	\begin{matrix}
		{\displaystyle \left[\frac{1}{P_w^{(s)}} \frac{\partial P_{w}^{(s)}}{\partial p_{vw}} + Z_{vw}\right]}
			&
			\text{if } t_{v}^{(s)} \in [t_{w}^{(s)}-\Delta-1, t_{w}^{(s)}-1] \text{ \& } v \in B(w)
			\\[2em]
			0 & \text{otherwise}
	\end{matrix}
	\right.
$$

In [81]:
S_set = twitts.tweet_id.tolist()

In [82]:
users_per_set = {}
for key in list_of_frames:
    users_per_set[key] = list_of_frames[key].user_id.unique().tolist()

In [83]:
def grad_comp(v,w):
    res = 0
    for S in S_set:
        w_act = activ_time(S, w)
        v_act = activ_time(S, v)
        if (w in graph_inverted) and (w in users_per_set[S]):
            if (v_act > w_act-DELTA-1) and (v_act < w_act-1):
                if v in graph_inverted[w]:
                    p = P(w,S)
                    ### this to be deleted,but for now we have values of P equals to zero
                    if p == 0:
                        p = 1
                    ###
                    res = res + 1/p * partial(S,v,w) + Z(S,v,w)
    return res
            

In [84]:
grad_comp(1616700401,1066926080)

0

### Gradient Computation

In [85]:
alpha = 0.001

In [ ]:
for i in edge_index:
    grad[edge_index[i]] = grad[edge_index[i]] + alpha*grad_comp(i[0], i[1])

In [ ]:
for i in grad:
    print i